In [148]:
# import packages

import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

In [149]:
menu = ['Fried Rice with Shrimp Dinner', 'Portobello Mushroom Jae Dinner', 'Goong Prik King Dinner', 'Vegetable Rolls', 'Por Tod', 'BKK Rolls', 'Summer Rolls', 'Satay', 'Tod Mon', 'BKK Dumplings', 'Yum Talay', 'Pla Goong', 'Peek Gai', 'Goong Bikini', 'Crispy Squid', 'Larb', 'Yum Nua', 'Num Tok', 'Papaya Salad', 'Tom Yum with Shrimps', 'Tom Yum with Vegetables', 'Tom Kha with Chicken', 'Tom Kha with Vegetables', 'Poe Tak', 'Tofu Soup', 'Goong Portobello Mushroom Dinner', 'Goong Portobello Mushroom Lunch', 'Volcano Seafood Dinner', 'Volcano Seafood Lunch', 'Nua Kata Dinner', 'Nua Kata Lunch', 'Talay Pow Dinner', 'Talay Pow Lunch', 'Red Curry Shrimp Dinner', 'Red Curry Shrimp Lunch', 'Nam Prik Ong Dinner', 'Nam Prik Ong Lunch', 'Goong Ka Tiem Dinner', 'Goong Ka Tiem Lunch', 'Goong Prik King Lunch', 'Talay Eggplant Dinner', 'Talay Eggplant Lunch', 'Talay Combo Dinner', 'Talay Combo Lunch', 'Goong Pad Pak Dinner', 'Goong Pad Pak Lunch', 'Panang Goong Dinner', 'Panang Goong Lunch', 'Goong Prik Prow Dinner', 'Goong Prik Prow Lunch', 'Ped Ka Prow Dinner', 'Ped Ka Prow Lunch', 'Gang Ped Yarng Dinner', 'Gang Ped Yarng Lunch', 'Ginger Rice Dinner', 'Ginger Rice Lunch', 'Bangkok Green Fried Rice Dinner', 'Bangkok Green Fried Rice Lunch', 'Roasted Duck on Rice Dinner', 'Roasted Duck on Rice Lunch', 'Fried Rice Dinner', 'Fried Rice Lunch', 'Fried Rice with Shrimp Lunch', 'Seafood Fried Rice Dinner', 'Seafood Fried Rice Lunch', 'Gai Prik Prow Dinner', 'Gai Prik Prow Lunch', 'Pa Ram Rong Srong Dinner', 'Pa Ram Rong Srong Lunch', 'Pad King Dinner', 'Pad King Lunch', 'Gai Siam Dinner', 'Gai Siam Lunch', 'Gai Him Ma Pan Dinner', 'Gai Him Ma Pan Lunch', 'Panang Gai Dinner', 'Panang Gai Lunch', 'Spiced Duck in Noodles Soup Dinner', 'Spiced Duck in Noodles Soup Lunch', 'Kaprow Dinner', 'Kaprow Lunch', 'Gai Eggplants Dinner', 'Gai Eggplants Lunch', 'Ka Tiem Dinner', 'Ka Tiem Lunch', 'Pad Prik King Dinner', 'Pad Prik King Lunch', 'Green Curry Chicken Dinner', 'Green Curry Chicken Lunch', 'Pad Thai Dinner', 'Pad Thai Lunch', 'Curry Noodles Dinner', 'Curry Noodles Lunch', 'Pad See-U Dinner', 'Pad See-U Lunch', 'Siam Noodles Dinner', 'Siam Noodles Lunch', 'Drunken Noodles Dinner', 'Drunken Noodles Lunch', 'Lad Na Dinner', 'Lad Na Lunch', 'Pad Thai Pak Dinner', 'Pad Thai Pak Lunch', 'Pad See-U Pak Dinner', 'Pad See-U Pak Lunch', 'Pa Ram Jae Dinner', 'Pa Ram Jae Lunch', 'Pad Eggplants Dinner', 'Pad Eggplants Lunch', 'Pad Pak Dinner', 'Pad Pak Lunch', 'Steam Garden Dinner', 'Steam Garden Lunch', 'Sweet and Sour Vegetables Dinner', 'Sweet and Sour Vegetables Lunch', 'Vegetable Fried Rice Dinner', 'Vegetable Fried Rice Lunch', 'Portobello Mushroom Jae Lunch', 'Lad Na Pak Dinner', 'Lad Na Pak Lunch', 'Drunken Noodles Pak Dinner', 'Drunken Noodles Pak Lunch', 'Kaprow Jae Dinner', 'Kaprow Jae Lunch', 'Panang Pak Dinner', 'Panang Pak Lunch', 'Gang Pak Dinner', 'Gang Pak Lunch', 'BKK Tofu Dinner', 'BKK Tofu Lunch', 'Sweet Sticky Rice with Mango', 'Coconut Custard with Taro Root', 'Palm Cake', 'Sticky Rice with Banana', 'Coconut Ice-Cream', 'Thai Iced Coffee', 'Grilled Chicken Salad', 'House Salad', 'Thai Iced Tea']

In [150]:
def menu_cleaner(menu):
    '''
    Clean and preprocess a menu.
    
    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split menus
    4. Remove English stopwords
    5. Rejoin to one string
    '''
    
    #1. Remove HTML tags
    menu = bs.BeautifulSoup(menu,features='lxml').text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', menu)
    
    #3. Remove punctuation
    menu = re.sub("[^a-zA-Z]", " ",menu)
    
    #4. Tokenize into words (all lower case)
    menu = menu.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    extra_stopwords = ['lunch','dinner','large','small','medium','kids', "thai", "chinese", "italian", "mediterranean",
                      "french", "japanese", "spanish", "greek", "ethiopian", "indian"]
    eng_stopwords.update(extra_stopwords)
    menu = [w for w in menu if not w in eng_stopwords]
    
    #6. Join the menu to one sentence
    menu = ' '.join(menu+emoticons)
    # add emoticons to the end

    return(menu)

In [151]:
%%time

num_items = len(menu)
menu_distinct = []
menu_clean = []
seen = set()

for i in range(0,num_items):
    if( (i+1)%500 == 0 ):
        # print progress
        print("Done with %d menus" %(i+1)) 
    clean_item = menu_cleaner(menu[i])
    if clean_item not in seen:
        seen.add(clean_item)
        menu_distinct.append(menu[i])
        menu_clean.append(clean_item)

CPU times: user 106 ms, sys: 12.3 ms, total: 118 ms
Wall time: 165 ms


In [152]:
menu_distinct

['Fried Rice with Shrimp Dinner',
 'Portobello Mushroom Jae Dinner',
 'Goong Prik King Dinner',
 'Vegetable Rolls',
 'Por Tod',
 'BKK Rolls',
 'Summer Rolls',
 'Satay',
 'Tod Mon',
 'BKK Dumplings',
 'Yum Talay',
 'Pla Goong',
 'Peek Gai',
 'Goong Bikini',
 'Crispy Squid',
 'Larb',
 'Yum Nua',
 'Num Tok',
 'Papaya Salad',
 'Tom Yum with Shrimps',
 'Tom Yum with Vegetables',
 'Tom Kha with Chicken',
 'Tom Kha with Vegetables',
 'Poe Tak',
 'Tofu Soup',
 'Goong Portobello Mushroom Dinner',
 'Volcano Seafood Dinner',
 'Nua Kata Dinner',
 'Talay Pow Dinner',
 'Red Curry Shrimp Dinner',
 'Nam Prik Ong Dinner',
 'Goong Ka Tiem Dinner',
 'Talay Eggplant Dinner',
 'Talay Combo Dinner',
 'Goong Pad Pak Dinner',
 'Panang Goong Dinner',
 'Goong Prik Prow Dinner',
 'Ped Ka Prow Dinner',
 'Gang Ped Yarng Dinner',
 'Ginger Rice Dinner',
 'Bangkok Green Fried Rice Dinner',
 'Roasted Duck on Rice Dinner',
 'Fried Rice Dinner',
 'Seafood Fried Rice Dinner',
 'Gai Prik Prow Dinner',
 'Pa Ram Rong Srong 

In [153]:
menu_clean

['fried rice shrimp',
 'portobello mushroom jae',
 'goong prik king',
 'vegetable rolls',
 'por tod',
 'bkk rolls',
 'summer rolls',
 'satay',
 'tod mon',
 'bkk dumplings',
 'yum talay',
 'pla goong',
 'peek gai',
 'goong bikini',
 'crispy squid',
 'larb',
 'yum nua',
 'num tok',
 'papaya salad',
 'tom yum shrimps',
 'tom yum vegetables',
 'tom kha chicken',
 'tom kha vegetables',
 'poe tak',
 'tofu soup',
 'goong portobello mushroom',
 'volcano seafood',
 'nua kata',
 'talay pow',
 'red curry shrimp',
 'nam prik ong',
 'goong ka tiem',
 'talay eggplant',
 'talay combo',
 'goong pad pak',
 'panang goong',
 'goong prik prow',
 'ped ka prow',
 'gang ped yarng',
 'ginger rice',
 'bangkok green fried rice',
 'roasted duck rice',
 'fried rice',
 'seafood fried rice',
 'gai prik prow',
 'pa ram rong srong',
 'pad king',
 'gai siam',
 'gai pan',
 'panang gai',
 'spiced duck noodles soup',
 'kaprow',
 'gai eggplants',
 'ka tiem',
 'pad prik king',
 'green curry chicken',
 'pad',
 'curry noodle

In [154]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

In [155]:
def find_pairs(noun, word_list):
    ret = []
    for word in word_list:
        ret.append(word + " " + noun)
    return ret
        

In [166]:
def add_hit_word(word, menu_item, word_to_menu, menu_to_word, seen_word):
    #we already know this word isn't a hit word
    if word in seen_word:
        return
    #this word was hit word for one other menu item
    elif word in word_to_menu:
        other_menu_item = word_to_menu[word]
        menu_to_word[other_menu_item].remove(word)
        del word_to_menu[word]
        seen_word.add(word)
    #this word isn't a hit word for any menu item yet
    else:
        menu_to_word[menu_item].append(word)
        word_to_menu[word] = menu_item       

In [171]:
wnl = WordNetLemmatizer()
is_noun = lambda pos: pos[:2] == 'NN'
word_to_menu = {}   #word: hit word, menu item
menu_to_word = {}   #key: menu item, value: hit words
seen_word = set()   #words that we've seen

for i in range(len(menu_clean)):
    menu_original = menu_distinct[i]
    menu_item = menu_clean[i]
    lemmatized_nouns = []
    lemmatized_other = []
    menu_to_word[menu_original] = []
    token_tag = pos_tag(menu_item.split())
    for pair in token_tag:
        if is_noun(pair[1]):
            #print(pair[0])
            res_noun = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
            lemmatized_nouns.append(res_noun)
        else:
            res_other = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
            lemmatized_other.append(res_other)
    for noun in lemmatized_nouns:
        add_hit_word(noun, menu_original, word_to_menu, menu_to_word, seen_word)
        #pair nouns with noun
        for noun_pair in find_pairs(noun, list(filter(lambda n: n != noun, lemmatized_nouns))):
            add_hit_word(noun_pair, menu_original, word_to_menu, menu_to_word, seen_word)
        #pair others with noun
        for other_pair in find_pairs(noun, lemmatized_other):  
            add_hit_word(other_pair, menu_original, word_to_menu, menu_to_word, seen_word)

In [172]:
menu_distinct
word_to_menu

{'banana': 'Sticky Rice with Banana',
 'banana rice': 'Sticky Rice with Banana',
 'bangkok': 'Bangkok Green Fried Rice Dinner',
 'bangkok rice': 'Bangkok Green Fried Rice Dinner',
 'bikini': 'Goong Bikini',
 'bkk dumpling': 'BKK Dumplings',
 'bkk roll': 'BKK Rolls',
 'bkk tofu': 'BKK Tofu Dinner',
 'cake palm': 'Palm Cake',
 'chicken curry': 'Green Curry Chicken Dinner',
 'chicken salad': 'Grilled Chicken Salad',
 'chicken tom': 'Tom Kha with Chicken',
 'coconut cream': 'Coconut Ice-Cream',
 'coconut custard': 'Coconut Custard with Taro Root',
 'coconut ice': 'Coconut Ice-Cream',
 'coconut root': 'Coconut Custard with Taro Root',
 'coconut taro': 'Coconut Custard with Taro Root',
 'coffee': 'Thai Iced Coffee',
 'combo': 'Talay Combo Dinner',
 'combo talay': 'Talay Combo Dinner',
 'cream': 'Coconut Ice-Cream',
 'cream coconut': 'Coconut Ice-Cream',
 'cream ice': 'Coconut Ice-Cream',
 'crispy': 'Crispy Squid',
 'crispy squid': 'Crispy Squid',
 'curry chicken': 'Green Curry Chicken Dinner

In [175]:
for hit_word in word_to_menu.keys():
    if hit_word in mike:
        print(word_to_menu[hit_word])
    








Tom Yum with Vegetables


In [87]:
%%time
# Lemmatizer

menu_clean_wnl = []

wnl = WordNetLemmatizer()

for i in range(0,num_items):
    if( (i+1)%500 == 0 ):
        # print progress
        print("Done with %d menus" %(i+1)) 
    
    wnl_stems = []
    token_tag = pos_tag(menu_clean[i].split())
    #print(token_tag)
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
        wnl_stems.append(res)
        
    
    menu_clean_wnl.append(' '.join(wnl_stems))

CPU times: user 73.2 ms, sys: 4.67 ms, total: 77.9 ms
Wall time: 85.1 ms


In [88]:
menu_clean_wnl

['fry rice shrimp',
 'portobello mushroom jae',
 'goong prik king',
 'vegetable roll',
 'por tod',
 'bkk roll',
 'summer roll',
 'satay',
 'tod mon',
 'bkk dumpling',
 'yum talay',
 'pla goong',
 'peek gai',
 'goong bikini',
 'crispy squid',
 'larb',
 'yum nua',
 'num tok',
 'papaya salad',
 'tom yum shrimp',
 'tom yum vegetable',
 'tom kha chicken',
 'tom kha vegetable',
 'poe tak',
 'tofu soup',
 'goong portobello mushroom',
 'goong portobello mushroom',
 'volcano seafood',
 'volcano seafood',
 'nua kata',
 'nua kata',
 'talay pow',
 'talay pow',
 'red curry shrimp',
 'red curry shrimp',
 'nam prik ong',
 'nam prik ong',
 'goong ka tiem',
 'goong ka tiem',
 'goong prik king',
 'talay eggplant',
 'talay eggplant',
 'talay combo',
 'talay combo',
 'goong pad pak',
 'goong pad pak',
 'panang goong',
 'panang goong',
 'goong prik prow',
 'goong prik prow',
 'ped ka prow',
 'ped ka prow',
 'gang ped yarng',
 'gang ped yarng',
 'ginger rice',
 'ginger rice',
 'bangkok green fry rice',
 'ba

In [14]:
#https://stackoverflow.com/questions/33587667/extracting-all-nouns-from-a-text-file-using-nltk
for i in range(len(menu_clean)):
    if menu_clean[i] != menu_clean_wnl[i]:
        print(menu_clean[i],"VS", menu_clean_wnl[i])
        

fried rice shrimp dinner VS fry rice shrimp dinner
vegetable rolls VS vegetable roll
bkk rolls VS bkk roll
summer rolls VS summer roll
bkk dumplings VS bkk dumpling
tom yum shrimps VS tom yum shrimp
tom yum vegetables VS tom yum vegetable
tom kha vegetables VS tom kha vegetable
bangkok green fried rice dinner VS bangkok green fry rice dinner
bangkok green fried rice lunch VS bangkok green fry rice lunch
roasted duck rice dinner VS roast duck rice dinner
roasted duck rice lunch VS roast duck rice lunch
fried rice dinner VS fry rice dinner
fried rice lunch VS fry rice lunch
fried rice shrimp lunch VS fry rice shrimp lunch
seafood fried rice dinner VS seafood fry rice dinner
seafood fried rice lunch VS seafood fry rice lunch
spiced duck noodles soup dinner VS spiced duck noodle soup dinner
spiced duck noodles soup lunch VS spiced duck noodle soup lunch
gai eggplants dinner VS gai eggplant dinner
gai eggplants lunch VS gai eggplant lunch
curry noodles dinner VS curry noodle dinner
curry no

In [22]:
lines = "gai eggplants pak dinner"
# function to test if something is a noun
is_noun = lambda pos: pos[:2] == 'NN'
# do the nlp stuff
tokenized = nltk.word_tokenize(lines)
nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 

print(nouns)

['gai', 'eggplants', 'dinner']


True
